# 準備

## ライブラリインポート

In [ ]:
import json
import base64
import os
from PIL import Image
import boto3
import botocore

boto3_bedrock = boto3.client(service_name='bedrock-runtime',region_name='us-east-1')


## util 関数

### 入力画像のバリデーション
[Anthropic Claude の inference parameter](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html#model-parameters-anthropic-claude-messages-request-response) では以下の制約がある

> data – (required) The base64 encoded image bytes for the image. The maximum image size is 3.75MB. The maximum height and width of an image is 8000 pixels. 

この制約を満たさないと以下のエラーが出る

> ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: messages.0.content.0.image.source.base64.data: Image format image/jpeg not supported

したがって base64 encode した画像サイズと最大の height/width が 8000 ピクセルより小さいかバリデーションする必要がある

In [ ]:
def is_image_valid(filepath):
    """Claude3 への入力に画像が有効か検査する

    Anthropic Claude の inference parameter の制約を満たすかチェックする

    Args:
        filepath(string): 入力画像への相対パス

    Returns:
        bool: valid なら True, invalid なら False

    Examples:

        関数の使い方

        >>> is_image_valid("path_to_image")
        True

    """
    file_size_bytes = os.path.getsize(filepath)
    file_size_mb = file_size_bytes/1000/1000
    if file_size_mb > 3.75:
        return False
    width, height = Image.open(filepath).size
    if (width > 8000) or (height > 8000):
        return False
    return True

### 一つの画像を入力とするプロンプト

In [ ]:
def invoke_claude_with_image(prompt_image_path,prompt_text):
    # Claude3 Haiku を使用する
    modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
    contentType = 'application/json'
    accept = 'application/json'
    outputText = "\n"
    if not is_image_valid(prompt_image_path):
        raise ValueError("input image invalid")
    with open(prompt_image_path, "rb") as image_file:
        image_bytes = image_file.read()
    encoded_image = base64.b64encode(image_bytes).decode("utf-8")
    body = json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 2000,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": "image/jpeg",
                                "data": encoded_image,
                            },
                        },
                        {"type": "text", "text": prompt_text},
                    ],
                }
            ],
        }
    )
    try:
        response = boto3_bedrock.invoke_model(body=body, modelId=modelId,contentType=contentType,accept=accept)
        response_body = json.loads(response.get('body').read())
        output_text = response_body.get('content')[0].get('text')
        return output_text

    except botocore.exceptions.ClientError as error:

        if error.response['Error']['Code'] == 'AccessDeniedException':
               print(f"\x1b[41m{error.response['Error']['Message']}\
                    \nTo troubeshoot this issue please refer to the following resources.\
                     \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                     \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

        else:
            raise error
